In [1]:
YEAR, POS, SHOT_POS = 2024, 72, 60
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
import math
from datetime import date
from datetime import timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait

#Import data
x = pd.read_csv('abbreviations.csv')
x.drop('Unnamed: 0', axis=1, inplace=True)
e1 = pd.read_pickle('Equation1.pkl')
e2 = pd.read_pickle('Equation2.pkl')
e3 = pd.read_pickle('Equation3.pkl')
e4 = pd.read_pickle('Equation4.pkl')
e5 = pd.read_pickle('Equation5.pkl')
e6 = pd.read_pickle('Equation6.pkl')
e7 = pd.read_pickle('Equation7.pkl')
e8 = pd.read_pickle('Equation8.pkl')
e9 = pd.read_pickle('Equation9.pkl')
e10 = pd.read_pickle('Equation10.pkl')
e11 = pd.read_pickle('Equation11.pkl')
e12 = pd.read_pickle('Equation12.pkl')
e13 = pd.read_pickle('Equation13.pkl')
e14 = pd.read_pickle('Equation14.pkl')
e15 = pd.read_pickle('Equation15.pkl')
e16 = pd.read_pickle('Equation16.pkl')
e17 = pd.read_pickle('Equation17.pkl')
e18 = pd.read_pickle('Equation18.pkl')
e19 = pd.read_pickle('Equation19.pkl')
e20 = pd.read_pickle('Equation20.pkl')
e21 = pd.read_pickle('Equation21.pkl')
e22 = pd.read_pickle('Equation22.pkl')
e23 = pd.read_pickle('Equation23.pkl')
e24 = pd.read_pickle('Equation24.pkl')
e25 = pd.read_pickle('Equation25.pkl')
e26 = pd.read_pickle('Equation26.pkl')
e27 = pd.read_pickle('Equation27.pkl')
e28 = pd.read_pickle('Equation28.pkl')
e29 = pd.read_pickle('Equation29.pkl')
e30 = pd.read_pickle('Equation30.pkl')

equations = [e1, e2, e3, e4, e5, e6, e7, e8, e9, e10, e11, e12, e13, e14, e15, e16, e17, e18, e19, e20, e21, e22, e23, e24, e25, e26, e27, e28, e29, e30]

#Get dates with missing scores
e1Missing = e1[pd.isna(e1['Home_Actual']) == True]
dailyscores = pd.DataFrame()
dates = e1Missing[pd.isna(e1Missing['Away_Actual']) == True].groupby('date')

#Itterate through missing dates
for name, group in dates:
    #Gets M, D, Y from name (date)
    d = re.findall('\d+', str(name))[:3]
    #Gets scores for date found
    s = 'https://www.sports-reference.com/cbb/boxscores/index.cgi?month=%s&day=%s&year=%s' % (d[1], d[2], d[0])
    url = requests.get(s)
    parse = BeautifulSoup(url.text, "lxml").prettify()
    start = [i for i in range(len(parse)) if parse.startswith("<table", i)]
    end = [i for i in range(len(parse)) if parse.startswith("</table", i)]
    #Itterate through row in missing games per date
    for i in range(0, len(start)):
        #Gets one Score, then prepares the data
        scores = pd.read_html(parse[start[i]:end[i]+8])[0]
        scores.columns = ['Team', 'Score', 'Drop']
        scores['Team'] = scores['Team'].astype(str)
        scores['Team'] = scores['Team'].apply(lambda x: re.split(' \(\d+\)', x)[0])
        scores['Team'] = scores['Team'].apply(lambda x: x[:-1] if x[-1:] == ' ' else x)
        scores.drop('Drop', axis = 1, inplace=True)
        scores['m'] = scores.iloc[-1, 1]
        scores = scores[(scores['Team'] != "Women's") & (scores['Team'] != "Men's")]
        scores = pd.concat([scores, scores.iloc[::-1].reset_index()], axis = 1)
        scores.columns = ['Away', 'Away_Score', 'Drop0', 'Drop1', 'Home', 'Home_Score', 'M']
        scores.drop(['Drop0', 'Drop1'], axis = 1, inplace=True)
        scores = scores[:-1]
        #Adds each individual score to a runnning chart for the day
        dailyscores = pd.concat([dailyscores, scores])

#Makes sure games are played today
for i, eq in enumerate(equations):
    if not dailyscores.empty:
        #For each formula and game, add the sportsref to the DF
        eq = eq.merge(x, how='left', left_on='Away', right_on='line')
        eq['Away_sportsref'] = eq['Away_sportsref'].fillna(eq['sportsref'])
        eq.drop(['team','abr','line','tor','hca','sportsref', 'sos'], axis = 1, inplace=True)
        eq.drop_duplicates(subset=['date', 'Away', 'Home'], inplace=True)
        eq = eq.merge(x, how='left', left_on='Home', right_on='line')
        eq['Home_sportsref'] = eq['Home_sportsref'].fillna(eq['sportsref'])
        eq.drop(['team','abr','line','tor','hca','sportsref', 'sos'], axis = 1, inplace=True)
        eq.drop_duplicates(subset=['date', 'Away', 'Home'], inplace=True)

        #Seperate missing and present data for each formula
        eqMissing = eq[pd.isna(eq['Home_Actual']) == True]
        eqPresent = eq[pd.isna(eq['Home_Actual']) == False]

        #Get just men's scores
        dailyscores = dailyscores[dailyscores['M'] == "Men's"]
        dailyscores.reset_index(inplace=True)
        dailyscores.drop('index', axis = 1, inplace = True)

        #For each formula and game, add the score to the DF
        eqMissing = eqMissing.merge(dailyscores, how='left', left_on=['Away_sportsref', 'Home_sportsref'], right_on=['Away', 'Home'])
        eqMissing.rename({'Away_x':'Away', 'Home_x':'Home'}, axis = 1, inplace = True)
        eqMissing['Away_Actual'] = eqMissing['Away_Actual'].fillna(eqMissing['Away_Score'])
        eqMissing['Home_Actual'] = eqMissing['Home_Actual'].fillna(eqMissing['Home_Score'])
        eqMissing.drop(['Away_y', 'Away_Score', 'Home_y', 'Home_Score', 'M'], axis = 1, inplace=True)
        eqMissing = eqMissing.merge(dailyscores, how='left', left_on=['Home_sportsref', 'Away_sportsref'], right_on=['Away', 'Home'])
        eqMissing.rename({'Away_x':'Away', 'Home_x':'Home'}, axis = 1, inplace = True)
        eqMissing['Away_Actual'] = eqMissing['Away_Actual'].fillna(eqMissing['Home_Score'])
        eqMissing['Home_Actual'] = eqMissing['Home_Actual'].fillna(eqMissing['Away_Score'])
        eqMissing.drop(['Away_y', 'Away_Score', 'Home_y', 'Home_Score', 'M'], axis = 1, inplace=True)

        #Fill in missing calculated fields, then re-combine missing and present
        eqMissing[['Actual_Spread', 'Home_Actual', 'Away_Actual']] = eqMissing[['Actual_Spread', 'Home_Actual', 'Away_Actual']].astype(float)
        eqMissing['Actual_Spread'] = eqMissing['Away_Actual'] - eqMissing['Home_Actual']
        eqMissing['User_Diff'] = abs(eqMissing['Predicted_Spread'] - eqMissing['Actual_Spread'])
        eqMissing['FD_Diff'] = abs(eqMissing['Home_Spread'].astype(float) - eqMissing['Actual_Spread'])
        eqMissing['More_Accurate'] = eqMissing['User_Diff'] < eqMissing['FD_Diff'] 
        result = [] 
        for index, row in eqMissing.iterrows():
            if row['Actual_Spread'] < 0:
                if float(row['Home_Spread']) < 0:
                    # Double Neg
                    if row['Actual_Spread'] < float(row['Home_Spread']):
                        result.append('Home')
                    elif row['Actual_Spread'] > float(row['Home_Spread']):
                        result.append('Away')
                    else:
                        result.append('Push')   
                else:
                    # Mine Neg FD Pos
                    result.append('Home')
            elif row['Actual_Spread'] > 0:
                if float(row['Home_Spread']) > 0:
                    # Double Pos
                    if row['Actual_Spread'] > float(row['Home_Spread']):
                        result.append('Away')
                    elif row['Actual_Spread'] < float(row['Home_Spread']):
                        result.append('Home') 
                    else:
                        result.append('Push')
                else:
                    # Mine Pos FD Neg
                    result.append('Away')
            else:
                if float(row['Home_Spread']) < 0:
                    result.append('Away')
                elif float(row['Home_Spread']) > 0:
                    result.append('Home')
                else: 
                    result.append('Push')
        eqMissing['Result'] = result
        eqMissing['Hit'] = eqMissing['bet'] == eqMissing['Result']
        eq = pd.concat([eqPresent, eqMissing])
        equations[i] = eq


#Scrape and prep Torvik Data
parse = BeautifulSoup(requests.get('https://www.barttorvik.com/#').text, "lxml").prettify()
tor = pd.read_html(parse[(parse.find("<table")):((parse.find("</table"))+8)])[0]
tor.columns = tor.columns.droplevel()
tor['AdjOE'] = tor['AdjOE'].str.split(' ').str[0]
tor['AdjDE'] = tor['AdjDE'].str.split(' ').str[0]
tor['Barthag'] = tor['Barthag'].str.split(' ').str[0]
tor['EFG%'] = tor['EFG%'].str.split(' ').str[0]
tor['EFGD%'] = tor['EFGD%'].str.split(' ').str[0]
tor['TOR'] = tor['TOR'].str.split(' ').str[0]
tor['TORD'] = tor['TORD'].str.split(' ').str[0]
tor['ORB'] = tor['ORB'].str.split(' ').str[0]
tor['DRB'] = tor['DRB'].str.split(' ').str[0]
tor['FTR'] = tor['FTR'].str.split(' ').str[0]
tor['FTRD'] = tor['FTRD'].str.split(' ').str[0]
tor['2P%'] = tor['2P%'].str.split(' ').str[0]
tor['2P%D'] = tor['2P%D'].str.split(' ').str[0]
tor['3P%'] = tor['3P%'].str.split(' ').str[0]
tor['3P%D'] = tor['3P%D'].str.split(' ').str[0]
tor['3PR'] = tor['3PR'].str.split(' ').str[0]
tor['3PRD'] = tor['3PRD'].str.split(' ').str[0]
tor['Adj T.'] = tor['Adj T.'].str.split(' ').str[0]
tor['WAB'] = tor['WAB'].str.split(' ').str[0]
tor['Team'] = tor['Team'].apply(lambda x: (x.split('  (H)')[0]) if '(H)' in x else x)
tor['Team'] = tor['Team'].apply(lambda x: (x.split('  (A)')[0]) if '(A)' in x else x)
tor.to_csv('torvik.csv')

#Scrape and prep offense Data
s = 'https://basketball.realgm.com/ncaa/team_stats/%s/Averages/Team_Totals/0' % YEAR
parse = BeautifulSoup(requests.get(s).text, "lxml").prettify()
off = pd.read_html(parse[(parse.find("<table")):((parse.find("</table"))+8)])[0]
off.drop(['#'], axis = 1, inplace = True)
off.to_csv('off.csv')

#Scrape and prep defense Data
s = 'https://basketball.realgm.com/ncaa/team-stats/%s/Averages/Opponent_Totals/0' % YEAR
parse = BeautifulSoup(requests.get(s).text, "lxml").prettify()
opp = pd.read_html(parse[(parse.find("<table")):((parse.find("</table"))+8)])[0]
opp.drop(['#'], axis = 1, inplace = True)
opp.to_csv('opp.csv')

#Scrape and prep Line data using Selenium
driver = webdriver.Firefox()
driver.get("https://www.sportsline.com/college-basketball/odds/")
wait = WebDriverWait(driver, 1)
driver.find_element("xpath", '/html/body/div[3]/div/div/div/button').click()

while True:
    if driver.find_element("xpath", '/html/body/div[1]/div[5]/div/section[1]/div/main/div/div[2]/section/div[3]/div[2]/button').is_enabled():
        driver.execute_script("arguments[0].click();", driver.find_element("xpath", '/html/body/div[1]/div[5]/div/section[1]/div/main/div/div[2]/section/div[3]/div[2]/button'))
    else:
        break

html = driver.page_source
driver.close()
parse = BeautifulSoup(html, 'lxml').prettify()
start = parse.find("<table")
end = parse.find("</table")
line = pd.read_html(parse[start:end+8])[0]
line = line[line['Matchup'] != line['consensus']]
line.drop('Proj Score', axis = 1, inplace=True)
line.dropna(inplace=True)
line.rename({'Unnamed: 3' : 'betmgm'}, axis = 1, inplace=True)
line['line'] = line['Matchup'].str.rsplit(' ', n = 1)
line['Record'] = line['line'].apply(lambda x: x[-1]) 
line['Team'] = line['line'].apply(lambda x: x[0]) 
line.drop(['line', 'Matchup'], axis = 1, inplace=True)
col = line.pop("Record")
line.insert(0, "Record", col)
col = line.pop("Team")
line.insert(0, "Team", col)
line['Team'] = line['Team'].astype(str)
line['Team'] = line['Team'].str[:-1]
line.reset_index(inplace=True)
line.drop('index', axis = 1, inplace=True)

#Join the line on itself to get home vs away matchups, then prep it
line = line[['Team', 'Record', 'fanduel']].astype(str)
line = line[line['Team'] != 'Georgi']
line = line[line['Team'] != 'Louisiana13-']
away = line.iloc[::2].reset_index()
away.rename({'Team':'Away', 'Record':'Away_Record', 'fanduel':'Away_fanduel'}, axis=1, inplace=True)
away.drop(['index'], axis=1, inplace=True)
away = away[away['Away_fanduel'] != '--']
away['Away_Spread'] = away['Away_fanduel'].apply(lambda x: (x.split(' ')[0]))
away['Away_odds'] = away['Away_fanduel'].apply(lambda x: (x.split(' ')[2]))
home = line.iloc[1: , :].iloc[::2].reset_index()
home.rename({'Team':'Home', 'Record':'Home_Record', 'fanduel':'Home_fanduel'}, axis=1, inplace=True)
line = pd.concat([away, home], axis=1)
line.drop(['index'], axis=1, inplace=True)
line = line[line['Home_fanduel'] != '--']
line['Home_Spread'] = line['Home_fanduel'].apply(lambda x: (x.split(' ')[0]))
line['Home_odds'] = line['Home_fanduel'].apply(lambda x: (x.split(' ')[2]))
line.drop(['Home_fanduel', 'Away_fanduel'], axis = 1, inplace=True)
line = x.merge(line, how='right', left_on='line', right_on='Away')
line.rename({'team':'Away_team', 'abr':'Away_abr', 'line':'Away_line', 'tor':'Away_tor', 'hca':'Away_hca', 'sportsref':'Away_sportsref', 'sos':'Away_sos'}, axis=1, inplace=True)
col = line.pop("Away_team")
line.insert(6, "Away_team", col)
col = line.pop("Away_abr")
line.insert(6, "Away_abr", col)
col = line.pop("Away_line")
line.insert(6, "Away_line", col)
col = line.pop("Away_tor")
line.insert(6, "Away_tor", col)
col = line.pop("Away_hca")
line.insert(6, "Away_hca", col)
col = line.pop("Away_sportsref")
line.insert(6, "Away_sportsref", col)
col = line.pop("Away_sos")
line.insert(6, "Away_sos", col)
line = x.merge(line, how='right', left_on='line', right_on='Home')
line.rename({'team':'Home_team', 'abr':'Home_abr', 'line':'Home_line', 'tor':'Home_tor', 'hca':'Home_hca', 'sportsref':'Home_sportsref', 'sos':'Home_sos'}, axis=1, inplace=True)
col = line.pop("Home_team")
line.insert(16, "Home_team", col)
col = line.pop("Home_abr")
line.insert(16, "Home_abr", col)
col = line.pop("Home_line")
line.insert(16, "Home_line", col)
col = line.pop("Home_tor")
line.insert(16, "Home_tor", col)
col = line.pop("Home_hca")
line.insert(16, "Home_hca", col)
col = line.pop("Home_sportsref")
line.insert(16, "Home_sportsref", col)
col = line.pop("Home_sos")
line.insert(16, "Home_sos", col)
line = line[(line['Home_team'] == line['Home_team']) & (line['Away_team'] == line['Away_team'])]
line.to_csv('line.csv')

#Open offense stats and merge with abbreviations
x = x.sort_values('team').reset_index()
off = pd.read_csv('off.csv')
off = off.merge(x, how='left', left_on='Team', right_on='team')
off.drop(['Unnamed: 0', 'team'], axis = 1, inplace=True)
off.insert(1, 'abr', off.pop('abr'))
off.insert(2, 'line', off.pop('line'))
off.insert(3, 'tor', off.pop('tor'))
off.sort_values(by='Team', ascending=True, inplace=True)

#Open defense stats and merge with abbreviations
opp = pd.read_csv('opp.csv')
opp.drop('Unnamed: 0', axis = 1, inplace=True)
opp = opp.sort_values(by='Team', ascending=True).reset_index()
stats = off.merge(opp, how='left', on='Team')
stats = stats.sort_values('Team').reset_index()
stats[['Team', 'abr', 'line', 'tor', 'sportsref', 'sos']] = x[['team', 'abr', 'line', 'tor', 'sportsref', 'sos']]

#Open torvik stats and merge with abbreviations
torvik = pd.read_csv('torvik.csv')
torvik.drop('Unnamed: 0', axis = 1, inplace=True)
torvik['Team'] = torvik['Team'].apply(lambda x: (x.split(' vs.')[0][:-1] if ' vs.' in x else x))
stats = stats.merge(torvik, how='left', left_on='tor', right_on='Team')
stats.drop('Team_y', axis = 1, inplace=True)
stats.rename({'Team_x':'team'}, axis=1, inplace=True)

#ste numerical columns to floats
stats[['Rk', 'G', 'AdjOE', 'AdjDE', 'Barthag', 'EFG%', 'EFGD%', 'TOR',
       'TORD', 'ORB', 'DRB', 'FTR', 'FTRD', '2P%', '2P%D', '3P%', '3P%D',
       '3PR', '3PRD', 'Adj T.', 'WAB']] = stats[[
       'Rk', 'G', 'AdjOE', 'AdjDE', 'Barthag', 'EFG%', 'EFGD%', 'TOR',
       'TORD', 'ORB', 'DRB', 'FTR', 'FTRD', '2P%', '2P%D', '3P%', '3P%D',
       '3PR', '3PRD', 'Adj T.', 'WAB']].astype(float)

#Add Homecourt Advantage
HCA = pd.read_csv('HCA.csv')
stats = stats.merge(HCA, how='left', left_on='hca', right_on='Team')
stats['HC_ADV'] = stats['HC_ADV'].fillna(3.2).astype(float)

#Add Strength of Schedule
SOS = pd.read_csv('sos.csv')
SOS.drop('Unnamed: 0', axis = 1, inplace = True)
stats = stats.merge(SOS, how='left', left_on='sos', right_on='SOSTeam')
stats.to_csv('stats.csv')

#FINAL CALCULATIONS
num = 0
away_conf = []
home_conf = []
for i, eq in enumerate(equations):
    num += 1
    away_score = []
    home_score = []

    for index, row in line.iterrows():
        a = stats[stats['line'] == row['Away']]
        # print("Away: " + row['Away'])
        h = stats[stats['line'] == row['Home']]
        # print("Home: " + row['Home'])

        if num == 1:
            h_pos = POS * (1 - ((((h.iloc[0]['TOR'] + a.iloc[0]['TORD']) / 2) - ((a.iloc[0]['TOR'] + h.iloc[0]['TORD']) / 2)) / 100)) * (1 - ((((a.iloc[0]['ORB'] + h.iloc[0]['DRB']) / 2) - ((h.iloc[0]['ORB'] + a.iloc[0]['DRB']) / 2)) / 100)) + ((((h.iloc[0]['3P%'] + a.iloc[0]['3P%D']) / 2) / 100) + (((h.iloc[0]['3PR'] + a.iloc[0]['3PRD']) / 2) / 100) * 1.5)
            a_pos = POS * (1 - ((((a.iloc[0]['TOR'] + h.iloc[0]['TORD']) / 2) - ((h.iloc[0]['TOR'] + a.iloc[0]['TORD']) / 2)) / 100)) * (1 - ((((h.iloc[0]['ORB'] + a.iloc[0]['DRB']) / 2) - ((a.iloc[0]['ORB'] + h.iloc[0]['DRB']) / 2)) / 100)) + ((((a.iloc[0]['3P%'] + h.iloc[0]['3P%D']) / 2) / 100) + (((a.iloc[0]['3PR'] + h.iloc[0]['3PRD']) / 2) / 100) * 1.5)
            home = ((((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 2) * h_pos) / 100) - ((h.iloc[0]['Rk'] - a.iloc[0]['Rk']) / 25)
            away = ((((a.iloc[0]['AdjOE'] + h.iloc[0]['AdjDE']) / 2) * a_pos) / 100) - ((a.iloc[0]['Rk'] - h.iloc[0]['Rk']) / 25)
            home += ((((h.iloc[0]['3P%'] + a.iloc[0]['3P%D']) / 2) / 100) + (((h.iloc[0]['3PR'] + a.iloc[0]['3PRD']) / 2) / 100) * 1.5)
            away += ((((a.iloc[0]['3P%'] + h.iloc[0]['3P%D']) / 2) / 100) + (((a.iloc[0]['3PR'] + h.iloc[0]['3PRD']) / 2) / 100) * 1.5)
            away_conf.append(a.iloc[0]['Conf'])
            home_conf.append(h.iloc[0]['Conf'])
            # print('Score: ' + a.iloc[0]['team'] + ' ' + str(round(away, 2)) + ' @ ' + h.iloc[0]['team'] + ' ' +  str(round(home, 2)) + ' TOTAL: ' + str(round(home + away, 2)))
        elif num == 2:
            h_pos = POS * (1 - ((((h.iloc[0]['TOR'] + a.iloc[0]['TORD']) / 2) - ((a.iloc[0]['TOR'] + h.iloc[0]['TORD']) / 2)) / 100)) * (1 - ((((a.iloc[0]['ORB'] + h.iloc[0]['DRB']) / 2) - ((h.iloc[0]['ORB'] + a.iloc[0]['DRB']) / 2)) / 100)) + ((((h.iloc[0]['3P%'] + a.iloc[0]['3P%D']) / 2) / 100) + (((h.iloc[0]['3PR'] + a.iloc[0]['3PRD']) / 2) / 100) * 1.5)
            a_pos = POS * (1 - ((((a.iloc[0]['TOR'] + h.iloc[0]['TORD']) / 2) - ((h.iloc[0]['TOR'] + a.iloc[0]['TORD']) / 2)) / 100)) * (1 - ((((h.iloc[0]['ORB'] + a.iloc[0]['DRB']) / 2) - ((a.iloc[0]['ORB'] + h.iloc[0]['DRB']) / 2)) / 100)) + ((((a.iloc[0]['3P%'] + h.iloc[0]['3P%D']) / 2) / 100) + (((a.iloc[0]['3PR'] + h.iloc[0]['3PRD']) / 2) / 100) * 1.5)
            home = ((((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 2) * h_pos) / 100) - ((h.iloc[0]['Rk'] - a.iloc[0]['Rk']) / 25)
            away = ((((a.iloc[0]['AdjOE'] + h.iloc[0]['AdjDE']) / 2) * a_pos) / 100) - ((a.iloc[0]['Rk'] - h.iloc[0]['Rk']) / 25)
        elif num == 3:
            h_pos = POS * (1 - ((((h.iloc[0]['TOR'] + a.iloc[0]['TORD']) / 2) - ((a.iloc[0]['TOR'] + h.iloc[0]['TORD']) / 2)) / 100)) * (1 - ((((a.iloc[0]['ORB'] + h.iloc[0]['DRB']) / 2) - ((h.iloc[0]['ORB'] + a.iloc[0]['DRB']) / 2)) / 100)) + ((((h.iloc[0]['3P%'] + a.iloc[0]['3P%D']) / 2) / 100) + (((h.iloc[0]['3PR'] + a.iloc[0]['3PRD']) / 2) / 100) * 1.5)
            a_pos = POS * (1 - ((((a.iloc[0]['TOR'] + h.iloc[0]['TORD']) / 2) - ((h.iloc[0]['TOR'] + a.iloc[0]['TORD']) / 2)) / 100)) * (1 - ((((h.iloc[0]['ORB'] + a.iloc[0]['DRB']) / 2) - ((a.iloc[0]['ORB'] + h.iloc[0]['DRB']) / 2)) / 100)) + ((((a.iloc[0]['3P%'] + h.iloc[0]['3P%D']) / 2) / 100) + (((a.iloc[0]['3PR'] + h.iloc[0]['3PRD']) / 2) / 100) * 1.5)
            home = (h_pos *  3 * (((((h.iloc[0]['3P%'] + a.iloc[0]['3P%D']) / 2) / 100) * (((h.iloc[0]['3PR'] + a.iloc[0]['3PRD']) / 2) / 100)))) + (h_pos *  1 * (((((h.iloc[0]['FTR'] + a.iloc[0]['FTRD']) / 2) / 100)))) + (h_pos * 2 * ((((h.iloc[0]['2P%'] + a.iloc[0]['2P%D']) / 2) / 100)) * (1 - ((((h.iloc[0]['FTR'] + a.iloc[0]['FTRD']) / (2 * 2)) / 100)) - ((((h.iloc[0]['3PR'] + a.iloc[0]['3PRD']) / 2) / 100)))) - ((h.iloc[0]['Rk'] - a.iloc[0]['Rk']) / 25) + h.iloc[0]['HC_ADV']
            away = (a_pos *  3 * (((((a.iloc[0]['3P%'] + h.iloc[0]['3P%D']) / 2) / 100) * (((a.iloc[0]['3PR'] + h.iloc[0]['3PRD']) / 2) / 100)))) + (a_pos *  1 * (((((a.iloc[0]['FTR'] + h.iloc[0]['FTRD']) / 2) / 100)))) + (a_pos * 2 * ((((a.iloc[0]['2P%'] + h.iloc[0]['2P%D']) / 2) / 100)) * (1 - ((((a.iloc[0]['FTR'] + h.iloc[0]['FTRD']) / (2 * 2)) / 100)) - ((((a.iloc[0]['3PR'] + h.iloc[0]['3PRD']) / 2) / 100)))) - ((a.iloc[0]['Rk'] - h.iloc[0]['Rk']) / 25)
        elif num == 4:
            home = ((((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 2) * POS) / 100) + h.iloc[0]['HC_ADV']
            away = ((((a.iloc[0]['AdjOE'] + h.iloc[0]['AdjDE']) / 2) * POS) / 100)
        elif num == 5:
            home =  (((((h.iloc[0]['3PR'] + a.iloc[0]['3PRD']) / 2) / 100) * (((h.iloc[0]['3P%'] + a.iloc[0]['3P%D']) / 2) / 100)) * (SHOT_POS * 3)) + ((((h.iloc[0]['FTR'] + a.iloc[0]['FTRD']) / (2)) / 100) * (h.iloc[0]['FT%_x']) * (SHOT_POS * 1)) + ((1 - (((h.iloc[0]['3PR'] + a.iloc[0]['3PRD']) / 2) / 100) + (((h.iloc[0]['FTR'] + a.iloc[0]['FTRD']) / (2*2)) / 100)) * (((h.iloc[0]['2P%'] + a.iloc[0]['2P%D']) / 2) / 100) * (SHOT_POS * 2))
            away = (((((a.iloc[0]['3PR'] + h.iloc[0]['3PRD']) / 2) / 100) * (((a.iloc[0]['3P%'] + h.iloc[0]['3P%D']) / 2) / 100)) * (SHOT_POS * 3)) + ((((a.iloc[0]['FTR'] + h.iloc[0]['FTRD']) / (2)) / 100) * (a.iloc[0]['FT%_x']) * (SHOT_POS * 1)) + ((1 - (((a.iloc[0]['3PR'] + h.iloc[0]['3PRD']) / 2) / 100) + (((a.iloc[0]['FTR'] + h.iloc[0]['FTRD']) / (2*2)) / 100)) * (((a.iloc[0]['2P%'] + h.iloc[0]['2P%D']) / 2) / 100) * (SHOT_POS * 2))
        elif num == 6:
            home =  (((((h.iloc[0]['3PR'] + a.iloc[0]['3PRD']) / 2) / 100) * (((h.iloc[0]['3P%'] + a.iloc[0]['3P%D']) / 2) / 100)) * (SHOT_POS * 3)) + ((((h.iloc[0]['FTR'] + a.iloc[0]['FTRD']) / (2)) / 100) * (h.iloc[0]['FT%_x']) * (SHOT_POS * 1)) + ((1 - (((h.iloc[0]['3PR'] + a.iloc[0]['3PRD']) / 2) / 100) + (((h.iloc[0]['FTR'] + a.iloc[0]['FTRD']) / (2*2)) / 100)) * (((h.iloc[0]['2P%'] + a.iloc[0]['2P%D']) / 2) / 100) * (SHOT_POS * 2)) + h.iloc[0]['HC_ADV'] + h.iloc[0]['SOS']
            away = (((((a.iloc[0]['3PR'] + h.iloc[0]['3PRD']) / 2) / 100) * (((a.iloc[0]['3P%'] + h.iloc[0]['3P%D']) / 2) / 100)) * (SHOT_POS * 3)) + ((((a.iloc[0]['FTR'] + h.iloc[0]['FTRD']) / (2)) / 100) * (a.iloc[0]['FT%_x']) * (SHOT_POS * 1)) + ((1 - (((a.iloc[0]['3PR'] + h.iloc[0]['3PRD']) / 2) / 100) + (((a.iloc[0]['FTR'] + h.iloc[0]['FTRD']) / (2*2)) / 100)) * (((a.iloc[0]['2P%'] + h.iloc[0]['2P%D']) / 2) / 100) * (SHOT_POS * 2)) + a.iloc[0]['SOS']
        elif num == 7:
            if abs(float(row['Home_Spread'])) <= 5:
                home =  (((((h.iloc[0]['3PR'] + a.iloc[0]['3PRD']) / 2) / 100) * (((h.iloc[0]['3P%'] + a.iloc[0]['3P%D']) / 2) / 100)) * (SHOT_POS * 3)) + ((((h.iloc[0]['FTR'] + a.iloc[0]['FTRD']) / (2)) / 100) * (h.iloc[0]['FT%_x']) * (SHOT_POS * 1)) + ((1 - (((h.iloc[0]['3PR'] + a.iloc[0]['3PRD']) / 2) / 100) + (((h.iloc[0]['FTR'] + a.iloc[0]['FTRD']) / (2*2)) / 100)) * (((h.iloc[0]['2P%'] + a.iloc[0]['2P%D']) / 2) / 100) * (SHOT_POS * 2)) + h.iloc[0]['HC_ADV'] + h.iloc[0]['SOS']
                away = (((((a.iloc[0]['3PR'] + h.iloc[0]['3PRD']) / 2) / 100) * (((a.iloc[0]['3P%'] + h.iloc[0]['3P%D']) / 2) / 100)) * (SHOT_POS * 3)) + ((((a.iloc[0]['FTR'] + h.iloc[0]['FTRD']) / (2)) / 100) * (a.iloc[0]['FT%_x']) * (SHOT_POS * 1)) + ((1 - (((a.iloc[0]['3PR'] + h.iloc[0]['3PRD']) / 2) / 100) + (((a.iloc[0]['FTR'] + h.iloc[0]['FTRD']) / (2*2)) / 100)) * (((a.iloc[0]['2P%'] + h.iloc[0]['2P%D']) / 2) / 100) * (SHOT_POS * 2)) + a.iloc[0]['SOS']
            elif (abs(float(row['Home_Spread'])) > 5) & (abs(float(row['Home_Spread'])) <= 10):
                home = ((((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 2) * POS) / 100) + h.iloc[0]['HC_ADV']
                away = ((((a.iloc[0]['AdjOE'] + h.iloc[0]['AdjDE']) / 2) * POS) / 100)
            else:
                h_pos = POS * (1 - ((((h.iloc[0]['TOR'] + a.iloc[0]['TORD']) / 2) - ((a.iloc[0]['TOR'] + h.iloc[0]['TORD']) / 2)) / 100)) * (1 - ((((a.iloc[0]['ORB'] + h.iloc[0]['DRB']) / 2) - ((h.iloc[0]['ORB'] + a.iloc[0]['DRB']) / 2)) / 100)) + ((((h.iloc[0]['3P%'] + a.iloc[0]['3P%D']) / 2) / 100) + (((h.iloc[0]['3PR'] + a.iloc[0]['3PRD']) / 2) / 100) * 1.5)
                a_pos = POS * (1 - ((((a.iloc[0]['TOR'] + h.iloc[0]['TORD']) / 2) - ((h.iloc[0]['TOR'] + a.iloc[0]['TORD']) / 2)) / 100)) * (1 - ((((h.iloc[0]['ORB'] + a.iloc[0]['DRB']) / 2) - ((a.iloc[0]['ORB'] + h.iloc[0]['DRB']) / 2)) / 100)) + ((((a.iloc[0]['3P%'] + h.iloc[0]['3P%D']) / 2) / 100) + (((a.iloc[0]['3PR'] + h.iloc[0]['3PRD']) / 2) / 100) * 1.5)
                home = ((((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 2) * h_pos) / 100) - ((h.iloc[0]['Rk'] - a.iloc[0]['Rk']) / 25)
                away = ((((a.iloc[0]['AdjOE'] + h.iloc[0]['AdjDE']) / 2) * a_pos) / 100) - ((a.iloc[0]['Rk'] - h.iloc[0]['Rk']) / 25)
                home += ((((h.iloc[0]['3P%'] + a.iloc[0]['3P%D']) / 2) / 100) + (((h.iloc[0]['3PR'] + a.iloc[0]['3PRD']) / 2) / 100) * 1.5)
                away += ((((a.iloc[0]['3P%'] + h.iloc[0]['3P%D']) / 2) / 100) + (((a.iloc[0]['3PR'] + h.iloc[0]['3PRD']) / 2) / 100) * 1.5)
        elif num == 8:
            if abs(float(row['Home_Spread'])) <= 5:
                home =  (((((h.iloc[0]['3PR'] + a.iloc[0]['3PRD']) / 2) / 100) * (((h.iloc[0]['3P%'] + a.iloc[0]['3P%D']) / 2) / 100)) * (SHOT_POS * 3)) + ((((h.iloc[0]['FTR'] + a.iloc[0]['FTRD']) / (2)) / 100) * (h.iloc[0]['FT%_x']) * (SHOT_POS * 1)) + ((1 - (((h.iloc[0]['3PR'] + a.iloc[0]['3PRD']) / 2) / 100) + (((h.iloc[0]['FTR'] + a.iloc[0]['FTRD']) / (2*2)) / 100)) * (((h.iloc[0]['2P%'] + a.iloc[0]['2P%D']) / 2) / 100) * (SHOT_POS * 2)) + h.iloc[0]['HC_ADV'] + h.iloc[0]['SOS']
                away = (((((a.iloc[0]['3PR'] + h.iloc[0]['3PRD']) / 2) / 100) * (((a.iloc[0]['3P%'] + h.iloc[0]['3P%D']) / 2) / 100)) * (SHOT_POS * 3)) + ((((a.iloc[0]['FTR'] + h.iloc[0]['FTRD']) / (2)) / 100) * (a.iloc[0]['FT%_x']) * (SHOT_POS * 1)) + ((1 - (((a.iloc[0]['3PR'] + h.iloc[0]['3PRD']) / 2) / 100) + (((a.iloc[0]['FTR'] + h.iloc[0]['FTRD']) / (2*2)) / 100)) * (((a.iloc[0]['2P%'] + h.iloc[0]['2P%D']) / 2) / 100) * (SHOT_POS * 2)) + a.iloc[0]['SOS']
            else:
                h_pos = POS * (1 - ((((h.iloc[0]['TOR'] + a.iloc[0]['TORD']) / 2) - ((a.iloc[0]['TOR'] + h.iloc[0]['TORD']) / 2)) / 100)) * (1 - ((((a.iloc[0]['ORB'] + h.iloc[0]['DRB']) / 2) - ((h.iloc[0]['ORB'] + a.iloc[0]['DRB']) / 2)) / 100)) + ((((h.iloc[0]['3P%'] + a.iloc[0]['3P%D']) / 2) / 100) + (((h.iloc[0]['3PR'] + a.iloc[0]['3PRD']) / 2) / 100) * 1.5)
                a_pos = POS * (1 - ((((a.iloc[0]['TOR'] + h.iloc[0]['TORD']) / 2) - ((h.iloc[0]['TOR'] + a.iloc[0]['TORD']) / 2)) / 100)) * (1 - ((((h.iloc[0]['ORB'] + a.iloc[0]['DRB']) / 2) - ((a.iloc[0]['ORB'] + h.iloc[0]['DRB']) / 2)) / 100)) + ((((a.iloc[0]['3P%'] + h.iloc[0]['3P%D']) / 2) / 100) + (((a.iloc[0]['3PR'] + h.iloc[0]['3PRD']) / 2) / 100) * 1.5)
                home = ((((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 2) * h_pos) / 100) - ((h.iloc[0]['Rk'] - a.iloc[0]['Rk']) / 25)
                away = ((((a.iloc[0]['AdjOE'] + h.iloc[0]['AdjDE']) / 2) * a_pos) / 100) - ((a.iloc[0]['Rk'] - h.iloc[0]['Rk']) / 25)
                home += ((((h.iloc[0]['3P%'] + a.iloc[0]['3P%D']) / 2) / 100) + (((h.iloc[0]['3PR'] + a.iloc[0]['3PRD']) / 2) / 100) * 1.5)
                away += ((((a.iloc[0]['3P%'] + h.iloc[0]['3P%D']) / 2) / 100) + (((a.iloc[0]['3PR'] + h.iloc[0]['3PRD']) / 2) / 100) * 1.5)
        elif num == 9:
            home = ((((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 2) * SHOT_POS) / 100) + h.iloc[0]['HC_ADV']
            away = ((((a.iloc[0]['AdjOE'] + h.iloc[0]['AdjDE']) / 2) * SHOT_POS) / 100)
        elif num == 10:
            home = ((((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 2) * SHOT_POS) / 100) + h.iloc[0]['HC_ADV'] + h.iloc[0]['SOS']
            away = ((((a.iloc[0]['AdjOE'] + h.iloc[0]['AdjDE']) / 2) * SHOT_POS) / 100) + a.iloc[0]['SOS']
        elif num == 11:
            home = (((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 4) * SHOT_POS) / 100
            away = (((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 4) * SHOT_POS) / 100
        elif num == 12:
            home = (((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 8) * SHOT_POS) / 100
            away = (((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 8) * SHOT_POS) / 100
        elif num == 13:
            home = ((((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 4) * SHOT_POS) / 100) + h.iloc[0]['HC_ADV'] + h.iloc[0]['SOS']
            away = ((((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 4) * SHOT_POS) / 100) + a.iloc[0]['SOS']
        elif num == 14:
            home = ((((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 8) * SHOT_POS) / 100) + h.iloc[0]['HC_ADV'] + h.iloc[0]['SOS']
            away = ((((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 8) * SHOT_POS) / 100) + a.iloc[0]['SOS']
        elif num == 15:
            home = ((((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 2) * SHOT_POS) / 100)
            away = ((((a.iloc[0]['AdjOE'] + h.iloc[0]['AdjDE']) / 2) * SHOT_POS) / 100)
        elif num == 16:
            h_pos = SHOT_POS * (1 - ((((h.iloc[0]['TOR'] + a.iloc[0]['TORD']) / 2) - ((a.iloc[0]['TOR'] + h.iloc[0]['TORD']) / 2)) / 100)) * (1 - ((((a.iloc[0]['ORB'] + h.iloc[0]['DRB']) / 2) - ((h.iloc[0]['ORB'] + a.iloc[0]['DRB']) / 2)) / 100)) + ((((h.iloc[0]['3P%'] + a.iloc[0]['3P%D']) / 2) / 100) + (((h.iloc[0]['3PR'] + a.iloc[0]['3PRD']) / 2) / 100) * 1.5)
            a_pos = SHOT_POS * (1 - ((((a.iloc[0]['TOR'] + h.iloc[0]['TORD']) / 2) - ((h.iloc[0]['TOR'] + a.iloc[0]['TORD']) / 2)) / 100)) * (1 - ((((h.iloc[0]['ORB'] + a.iloc[0]['DRB']) / 2) - ((a.iloc[0]['ORB'] + h.iloc[0]['DRB']) / 2)) / 100)) + ((((a.iloc[0]['3P%'] + h.iloc[0]['3P%D']) / 2) / 100) + (((a.iloc[0]['3PR'] + h.iloc[0]['3PRD']) / 2) / 100) * 1.5)
            home =  (((((h.iloc[0]['3PR'] + a.iloc[0]['3PRD']) / 2) / 100) * (((h.iloc[0]['3P%'] + a.iloc[0]['3P%D']) / 2) / 100)) * (h_pos * 3)) + ((((h.iloc[0]['FTR'] + a.iloc[0]['FTRD']) / (2)) / 100) * (h.iloc[0]['FT%_x']) * (h_pos * 1)) + ((1 - (((h.iloc[0]['3PR'] + a.iloc[0]['3PRD']) / 2) / 100) + (((h.iloc[0]['FTR'] + a.iloc[0]['FTRD']) / (2*2)) / 100)) * (((h.iloc[0]['2P%'] + a.iloc[0]['2P%D']) / 2) / 100) * (h_pos * 2)) + h.iloc[0]['HC_ADV'] + h.iloc[0]['SOS']
            away = (((((a.iloc[0]['3PR'] + h.iloc[0]['3PRD']) / 2) / 100) * (((a.iloc[0]['3P%'] + h.iloc[0]['3P%D']) / 2) / 100)) * (a_pos * 3)) + ((((a.iloc[0]['FTR'] + h.iloc[0]['FTRD']) / (2)) / 100) * (a.iloc[0]['FT%_x']) * (a_pos * 1)) + ((1 - (((a.iloc[0]['3PR'] + h.iloc[0]['3PRD']) / 2) / 100) + (((a.iloc[0]['FTR'] + h.iloc[0]['FTRD']) / (2*2)) / 100)) * (((a.iloc[0]['2P%'] + h.iloc[0]['2P%D']) / 2) / 100) * (a_pos * 2)) + a.iloc[0]['SOS']
        elif num == 17:
            h_pos = SHOT_POS * (1 - ((((h.iloc[0]['TOR'] + a.iloc[0]['TORD'])) - ((a.iloc[0]['TOR'] + h.iloc[0]['TORD']))) / 100))
            a_pos = SHOT_POS * (1 - ((((a.iloc[0]['TOR'] + h.iloc[0]['TORD'])) - ((h.iloc[0]['TOR'] + a.iloc[0]['TORD']))) / 100))
            home =  (((((h.iloc[0]['3PR'] + a.iloc[0]['3PRD']) / 2) / 100) * (((h.iloc[0]['3P%'] + a.iloc[0]['3P%D']) / 2) / 100)) * (h_pos * 3)) + ((((h.iloc[0]['FTR'] + a.iloc[0]['FTRD']) / (2)) / 100) * (h.iloc[0]['FT%_x']) * (h_pos * 1)) + ((1 - (((h.iloc[0]['3PR'] + a.iloc[0]['3PRD']) / 2) / 100) + (((h.iloc[0]['FTR'] + a.iloc[0]['FTRD']) / (2*2)) / 100)) * (((h.iloc[0]['2P%'] + a.iloc[0]['2P%D']) / 2) / 100) * (h_pos * 2)) + h.iloc[0]['HC_ADV'] + h.iloc[0]['SOS']
            away = (((((a.iloc[0]['3PR'] + h.iloc[0]['3PRD']) / 2) / 100) * (((a.iloc[0]['3P%'] + h.iloc[0]['3P%D']) / 2) / 100)) * (a_pos * 3)) + ((((a.iloc[0]['FTR'] + h.iloc[0]['FTRD']) / (2)) / 100) * (a.iloc[0]['FT%_x']) * (a_pos * 1)) + ((1 - (((a.iloc[0]['3PR'] + h.iloc[0]['3PRD']) / 2) / 100) + (((a.iloc[0]['FTR'] + h.iloc[0]['FTRD']) / (2*2)) / 100)) * (((a.iloc[0]['2P%'] + h.iloc[0]['2P%D']) / 2) / 100) * (a_pos * 2)) + a.iloc[0]['SOS']     
        elif num == 18:
            h_pos = SHOT_POS * (1 - ((((a.iloc[0]['ORB'] + h.iloc[0]['DRB'])) - ((h.iloc[0]['ORB'] + a.iloc[0]['DRB']))) / 100))
            a_pos = SHOT_POS * (1 - ((((h.iloc[0]['ORB'] + a.iloc[0]['DRB'])) - ((a.iloc[0]['ORB'] + h.iloc[0]['DRB']))) / 100))
            home =  (((((h.iloc[0]['3PR'] + a.iloc[0]['3PRD']) / 2) / 100) * (((h.iloc[0]['3P%'] + a.iloc[0]['3P%D']) / 2) / 100)) * (h_pos * 3)) + ((((h.iloc[0]['FTR'] + a.iloc[0]['FTRD']) / (2)) / 100) * (h.iloc[0]['FT%_x']) * (h_pos * 1)) + ((1 - (((h.iloc[0]['3PR'] + a.iloc[0]['3PRD']) / 2) / 100) + (((h.iloc[0]['FTR'] + a.iloc[0]['FTRD']) / (2*2)) / 100)) * (((h.iloc[0]['2P%'] + a.iloc[0]['2P%D']) / 2) / 100) * (h_pos * 2)) + h.iloc[0]['HC_ADV'] + h.iloc[0]['SOS']
            away = (((((a.iloc[0]['3PR'] + h.iloc[0]['3PRD']) / 2) / 100) * (((a.iloc[0]['3P%'] + h.iloc[0]['3P%D']) / 2) / 100)) * (a_pos * 3)) + ((((a.iloc[0]['FTR'] + h.iloc[0]['FTRD']) / (2)) / 100) * (a.iloc[0]['FT%_x']) * (a_pos * 1)) + ((1 - (((a.iloc[0]['3PR'] + h.iloc[0]['3PRD']) / 2) / 100) + (((a.iloc[0]['FTR'] + h.iloc[0]['FTRD']) / (2*2)) / 100)) * (((a.iloc[0]['2P%'] + h.iloc[0]['2P%D']) / 2) / 100) * (a_pos * 2)) + a.iloc[0]['SOS']     
        elif num == 19:
            h_pos = SHOT_POS * ((((h.iloc[0]['3P%'] + a.iloc[0]['3P%D'])) / 100) + (((h.iloc[0]['3PR'] + a.iloc[0]['3PRD'])) / 100))
            a_pos = SHOT_POS * ((((a.iloc[0]['3P%'] + h.iloc[0]['3P%D'])) / 100) + (((a.iloc[0]['3PR'] + h.iloc[0]['3PRD'])) / 100))
            home =  (((((h.iloc[0]['3PR'] + a.iloc[0]['3PRD']) / 2) / 100) * (((h.iloc[0]['3P%'] + a.iloc[0]['3P%D']) / 2) / 100)) * (h_pos * 3)) + ((((h.iloc[0]['FTR'] + a.iloc[0]['FTRD']) / (2)) / 100) * (h.iloc[0]['FT%_x']) * (h_pos * 1)) + ((1 - (((h.iloc[0]['3PR'] + a.iloc[0]['3PRD']) / 2) / 100) + (((h.iloc[0]['FTR'] + a.iloc[0]['FTRD']) / (2*2)) / 100)) * (((h.iloc[0]['2P%'] + a.iloc[0]['2P%D']) / 2) / 100) * (h_pos * 2)) + h.iloc[0]['HC_ADV'] + h.iloc[0]['SOS']
            away = (((((a.iloc[0]['3PR'] + h.iloc[0]['3PRD']) / 2) / 100) * (((a.iloc[0]['3P%'] + h.iloc[0]['3P%D']) / 2) / 100)) * (a_pos * 3)) + ((((a.iloc[0]['FTR'] + h.iloc[0]['FTRD']) / (2)) / 100) * (a.iloc[0]['FT%_x']) * (a_pos * 1)) + ((1 - (((a.iloc[0]['3PR'] + h.iloc[0]['3PRD']) / 2) / 100) + (((a.iloc[0]['FTR'] + h.iloc[0]['FTRD']) / (2*2)) / 100)) * (((a.iloc[0]['2P%'] + h.iloc[0]['2P%D']) / 2) / 100) * (a_pos * 2)) + a.iloc[0]['SOS']     
        elif num == 20:
            if abs(float(row['Home_Spread'])) <= 5:
                home =  (((((h.iloc[0]['3PR'] + a.iloc[0]['3PRD']) / 2) / 100) * (((h.iloc[0]['3P%'] + a.iloc[0]['3P%D']) / 2) / 100)) * (SHOT_POS * 3)) + ((((h.iloc[0]['FTR'] + a.iloc[0]['FTRD']) / (2)) / 100) * (h.iloc[0]['FT%_x']) * (SHOT_POS * 1)) + ((1 - (((h.iloc[0]['3PR'] + a.iloc[0]['3PRD']) / 2) / 100) + (((h.iloc[0]['FTR'] + a.iloc[0]['FTRD']) / (2*2)) / 100)) * (((h.iloc[0]['2P%'] + a.iloc[0]['2P%D']) / 2) / 100) * (SHOT_POS * 2)) + h.iloc[0]['HC_ADV'] + h.iloc[0]['SOS']
                away = (((((a.iloc[0]['3PR'] + h.iloc[0]['3PRD']) / 2) / 100) * (((a.iloc[0]['3P%'] + h.iloc[0]['3P%D']) / 2) / 100)) * (SHOT_POS * 3)) + ((((a.iloc[0]['FTR'] + h.iloc[0]['FTRD']) / (2)) / 100) * (a.iloc[0]['FT%_x']) * (SHOT_POS * 1)) + ((1 - (((a.iloc[0]['3PR'] + h.iloc[0]['3PRD']) / 2) / 100) + (((a.iloc[0]['FTR'] + h.iloc[0]['FTRD']) / (2*2)) / 100)) * (((a.iloc[0]['2P%'] + h.iloc[0]['2P%D']) / 2) / 100) * (SHOT_POS * 2)) + a.iloc[0]['SOS']
            elif (abs(float(row['Home_Spread'])) > 5) & (abs(float(row['Home_Spread'])) <= 10):
                home = ((((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 2) * SHOT_POS) / 100)
                away = ((((a.iloc[0]['AdjOE'] + h.iloc[0]['AdjDE']) / 2) * SHOT_POS) / 100)
            else:
                h_pos = SHOT_POS * (1 - ((((h.iloc[0]['TOR'] + a.iloc[0]['TORD']) / 2) - ((a.iloc[0]['TOR'] + h.iloc[0]['TORD']) / 2)) / 100)) * (1 - ((((a.iloc[0]['ORB'] + h.iloc[0]['DRB']) / 2) - ((h.iloc[0]['ORB'] + a.iloc[0]['DRB']) / 2)) / 100)) + ((((h.iloc[0]['3P%'] + a.iloc[0]['3P%D']) / 2) / 100) + (((h.iloc[0]['3PR'] + a.iloc[0]['3PRD']) / 2) / 100) * 1.5)
                a_pos = SHOT_POS * (1 - ((((a.iloc[0]['TOR'] + h.iloc[0]['TORD']) / 2) - ((h.iloc[0]['TOR'] + a.iloc[0]['TORD']) / 2)) / 100)) * (1 - ((((h.iloc[0]['ORB'] + a.iloc[0]['DRB']) / 2) - ((a.iloc[0]['ORB'] + h.iloc[0]['DRB']) / 2)) / 100)) + ((((a.iloc[0]['3P%'] + h.iloc[0]['3P%D']) / 2) / 100) + (((a.iloc[0]['3PR'] + h.iloc[0]['3PRD']) / 2) / 100) * 1.5)
                home =  (((((h.iloc[0]['3PR'] + a.iloc[0]['3PRD']) / 2) / 100) * (((h.iloc[0]['3P%'] + a.iloc[0]['3P%D']) / 2) / 100)) * (h_pos * 3)) + ((((h.iloc[0]['FTR'] + a.iloc[0]['FTRD']) / (2)) / 100) * (h.iloc[0]['FT%_x']) * (h_pos * 1)) + ((1 - (((h.iloc[0]['3PR'] + a.iloc[0]['3PRD']) / 2) / 100) + (((h.iloc[0]['FTR'] + a.iloc[0]['FTRD']) / (2*2)) / 100)) * (((h.iloc[0]['2P%'] + a.iloc[0]['2P%D']) / 2) / 100) * (h_pos * 2)) + h.iloc[0]['HC_ADV'] + h.iloc[0]['SOS']
                away = (((((a.iloc[0]['3PR'] + h.iloc[0]['3PRD']) / 2) / 100) * (((a.iloc[0]['3P%'] + h.iloc[0]['3P%D']) / 2) / 100)) * (a_pos * 3)) + ((((a.iloc[0]['FTR'] + h.iloc[0]['FTRD']) / (2)) / 100) * (a.iloc[0]['FT%_x']) * (a_pos * 1)) + ((1 - (((a.iloc[0]['3PR'] + h.iloc[0]['3PRD']) / 2) / 100) + (((a.iloc[0]['FTR'] + h.iloc[0]['FTRD']) / (2*2)) / 100)) * (((a.iloc[0]['2P%'] + h.iloc[0]['2P%D']) / 2) / 100) * (a_pos * 2)) + a.iloc[0]['SOS']
        elif num == 21:
            if abs(float(row['Home_Spread'])) <= 5:   
                home = ((((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 4) * SHOT_POS) / 100) + a.iloc[0]['SOS']
                away = ((((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 4) * SHOT_POS) / 100) + h.iloc[0]['HC_ADV'] + h.iloc[0]['SOS']
            elif (abs(float(row['Home_Spread'])) > 5) & (abs(float(row['Home_Spread'])) <= 10):
                home = ((((a.iloc[0]['AdjOE'] + h.iloc[0]['AdjDE']) / 2) * SHOT_POS) / 100) + a.iloc[0]['SOS']
                away = ((((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 2) * SHOT_POS) / 100) + h.iloc[0]['HC_ADV'] + h.iloc[0]['SOS']
            else:
                home = (((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 4) * SHOT_POS) / 100
                away = (((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 4) * SHOT_POS) / 100
        elif num == 22:
            home = (((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 4) * SHOT_POS) / 100
            away = (((a.iloc[0]['AdjOE'] + h.iloc[0]['AdjDE']) / 4) * SHOT_POS) / 100
        elif num == 23:
            home = (((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 8) * SHOT_POS) / 100
            away = (((a.iloc[0]['AdjOE'] + h.iloc[0]['AdjDE']) / 8) * SHOT_POS) / 100
        elif num == 24:
            home = ((((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 4) * SHOT_POS) / 100) + h.iloc[0]['HC_ADV'] + h.iloc[0]['SOS']
            away = ((((a.iloc[0]['AdjOE'] + h.iloc[0]['AdjDE']) / 4) * SHOT_POS) / 100) + a.iloc[0]['SOS']
        elif num == 25:
            home = ((((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 8) * SHOT_POS) / 100) + h.iloc[0]['HC_ADV'] + h.iloc[0]['SOS']
            away = ((((a.iloc[0]['AdjOE'] + h.iloc[0]['AdjDE']) / 8) * SHOT_POS) / 100) + a.iloc[0]['SOS']
        elif num == 26:
            if abs(float(row['Home_Spread'])) <= 5:   
                home = ((((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 4) * SHOT_POS) / 100) + a.iloc[0]['SOS'] + h.iloc[0]['HC_ADV']
                away = ((((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 4) * SHOT_POS) / 100) + h.iloc[0]['SOS']
            elif (abs(float(row['Home_Spread'])) > 5) & (abs(float(row['Home_Spread'])) <= 10):
                home = ((((a.iloc[0]['AdjOE'] + h.iloc[0]['AdjDE']) / 2) * SHOT_POS) / 100) + a.iloc[0]['SOS'] + h.iloc[0]['HC_ADV']
                away = ((((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 2) * SHOT_POS) / 100) + h.iloc[0]['SOS']
            else:
                home = (((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 4) * SHOT_POS) / 100
                away = (((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 4) * SHOT_POS) / 100
        elif num == 27:
            if abs(float(row['Home_Spread'])) <= 5:   
                home = ((((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 4) * SHOT_POS) / 100) +  h.iloc[0]['HC_ADV']
                away = ((((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 4) * SHOT_POS) / 100)
            elif (abs(float(row['Home_Spread'])) > 5) & (abs(float(row['Home_Spread'])) <= 10):
                home = ((((a.iloc[0]['AdjOE'] + h.iloc[0]['AdjDE']) / 2) * SHOT_POS) / 100) + h.iloc[0]['HC_ADV']
                away = ((((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 2) * SHOT_POS) / 100)
            else:
                home = (((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 4) * SHOT_POS) / 100
                away = (((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 4) * SHOT_POS) / 100
        elif num == 28:
            if abs(float(row['Home_Spread'])) <= 5:   
                home = ((((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 4) * SHOT_POS) / 100) + a.iloc[0]['SOS']
                away = ((((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 4) * SHOT_POS) / 100) + h.iloc[0]['SOS']
            elif (abs(float(row['Home_Spread'])) > 5) & (abs(float(row['Home_Spread'])) <= 10):
                home = ((((a.iloc[0]['AdjOE'] + h.iloc[0]['AdjDE']) / 2) * SHOT_POS) / 100) + a.iloc[0]['SOS']
                away = ((((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 2) * SHOT_POS) / 100) + h.iloc[0]['SOS']
            else:
                home = (((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 4) * SHOT_POS) / 100
                away = (((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 4) * SHOT_POS) / 100
        elif num == 29:
            if abs(float(row['Home_Spread'])) <= 5:   
                home = ((((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 4) * SHOT_POS) / 100)
                away = ((((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 4) * SHOT_POS) / 100)
            elif (abs(float(row['Home_Spread'])) > 5) & (abs(float(row['Home_Spread'])) <= 10):
                home = ((((a.iloc[0]['AdjOE'] + h.iloc[0]['AdjDE']) / 2) * SHOT_POS) / 100)
                away = ((((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 2) * SHOT_POS) / 100)
            else:
                home = (((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 4) * SHOT_POS) / 100
                away = (((h.iloc[0]['AdjOE'] + a.iloc[0]['AdjDE']) / 4) * SHOT_POS) / 100
        
                

        away_score.append(round(away, 2))
        home_score.append(round(home, 2))
    
    line['Predicted_Away_Score'] = away_score
    line['Predicted_Home_Score'] = home_score
    line['Predicted_Spread'] = -(line['Predicted_Home_Score'] - line['Predicted_Away_Score'])

    bet = []
    bet_by = []
    for index, row in line.iterrows():
        bet_by.append(abs(float(row['Home_Spread']) - float(row['Predicted_Spread'])))
        if row['Predicted_Spread'] < 0:
            if float(row['Home_Spread']) < 0:
                # Double Neg
                if row['Predicted_Spread'] < float(row['Home_Spread']):
                    bet.append('Home')
                elif row['Predicted_Spread'] > float(row['Home_Spread']):
                    bet.append('Away')
                else:
                    bet.append('Push')   
            else:
                # Mine Neg FD Pos
                bet.append('Home')
        elif row['Predicted_Spread'] > 0:
            if float(row['Home_Spread']) > 0:
                # Double Pos
                if row['Predicted_Spread'] > float(row['Home_Spread']):
                    bet.append('Away')
                elif row['Predicted_Spread'] < float(row['Home_Spread']):
                    bet.append('Home') 
                else:
                    bet.append('Push')
            else:
                # Mine Pos FD Neg
                bet.append('Away')
        else:
            if float(row['Home_Spread']) < 0:
                bet.append('Away')
            elif float(row['Home_Spread']) > 0:
                bet.append('Home')
            else: 
                bet.append('Push')

        
    if num == 1:
        line.insert(7, 'Away_Conf', away_conf)
        line.insert(17, 'Home_Conf', home_conf)
        line.insert(0, 'date', date.today())
        
    line['bet'] = bet
    line['bet_by'] = bet_by
    line[['Away_Actual', 'Home_Actual', 'Actual_Spread', 'User_Diff', 'FD_Diff']] = np.nan
    line['date'] = date.today()

    if not eq.empty:
        eq['date'] = eq.apply(lambda x: (x['date'] + timedelta(days = 1)) if x['Away_Actual'] != x['Away_Actual'] else x['date'], axis = 1)
        eq['date'] = eq.apply(lambda x: date.today() if ((x['Away_Actual']!=x['Away_Actual']) & (x['date'] > date.today())) else x['date'], axis = 1)
    eq = pd.concat([eq, line])
    eq.drop_duplicates(subset=['date', 'Away', 'Home'], keep = 'last', inplace=True)
    equations[i] = eq

    

equations[0].to_pickle('Equation1.pkl')
equations[1].to_pickle('Equation2.pkl')
equations[2].to_pickle('Equation3.pkl')
equations[3].to_pickle('Equation4.pkl')
equations[4].to_pickle('Equation5.pkl')
equations[5].to_pickle('Equation6.pkl')
equations[6].to_pickle('Equation7.pkl')
equations[7].to_pickle('Equation8.pkl')
equations[8].to_pickle('Equation9.pkl')
equations[9].to_pickle('Equation10.pkl')
equations[10].to_pickle('Equation11.pkl')
equations[11].to_pickle('Equation12.pkl')
equations[12].to_pickle('Equation13.pkl')
equations[13].to_pickle('Equation14.pkl')
equations[14].to_pickle('Equation15.pkl')
equations[15].to_pickle('Equation16.pkl')
equations[16].to_pickle('Equation17.pkl')
equations[17].to_pickle('Equation18.pkl')
equations[18].to_pickle('Equation19.pkl')
equations[19].to_pickle('Equation20.pkl')
equations[20].to_pickle('Equation21.pkl')
equations[21].to_pickle('Equation22.pkl')
equations[22].to_pickle('Equation23.pkl')
equations[23].to_pickle('Equation24.pkl')
equations[24].to_pickle('Equation25.pkl')
equations[25].to_pickle('Equation26.pkl')
equations[26].to_pickle('Equation27.pkl')
equations[27].to_pickle('Equation28.pkl')
equations[28].to_pickle('Equation29.pkl')
equations[29].to_pickle('Equation30.pkl')


equations[0].to_pickle('SecondaryEquation1.pkl')
equations[1].to_pickle('SecondaryEquation2.pkl')
equations[2].to_pickle('SecondaryEquation3.pkl')
equations[3].to_pickle('SecondaryEquation4.pkl')
equations[4].to_pickle('SecondaryEquation5.pkl')
equations[5].to_pickle('SecondaryEquation6.pkl')
equations[6].to_pickle('SecondaryEquation7.pkl')
equations[7].to_pickle('SecondaryEquation8.pkl')
equations[8].to_pickle('SecondaryEquation9.pkl')
equations[9].to_pickle('SecondaryEquation10.pkl')
equations[10].to_pickle('SecondaryEquation11.pkl')
equations[11].to_pickle('SecondaryEquation12.pkl')
equations[12].to_pickle('SecondaryEquation13.pkl')
equations[13].to_pickle('SecondaryEquation14.pkl')
equations[14].to_pickle('SecondaryEquation15.pkl')
equations[15].to_pickle('SecondaryEquation16.pkl')
equations[16].to_pickle('SecondaryEquation17.pkl')
equations[17].to_pickle('SecondaryEquation18.pkl')
equations[18].to_pickle('SecondaryEquation19.pkl')
equations[19].to_pickle('SecondaryEquation20.pkl')
equations[20].to_pickle('SecondaryEquation21.pkl')
equations[21].to_pickle('SecondaryEquation22.pkl')
equations[22].to_pickle('SecondaryEquation23.pkl')
equations[23].to_pickle('SecondaryEquation24.pkl')
equations[24].to_pickle('SecondaryEquation25.pkl')
equations[25].to_pickle('SecondaryEquation26.pkl')
equations[26].to_pickle('SecondaryEquation27.pkl')
equations[27].to_pickle('SecondaryEquation28.pkl')
equations[28].to_pickle('SecondaryEquation29.pkl')
equations[29].to_pickle('SecondaryEquation30.pkl')

equations[29]

,date,Away,Away_team,Away_abr,Away_line,Away_tor,Away_hca,Away_sportsref,Away_sos,Away_Conf,...,bet,bet_by,Away_Actual,Home_Actual,Actual_Spread,User_Diff,FD_Diff,More_Accurate,Result,Hit
0,2024-02-03,Arkansas,Arkansas,ARK,Arkansas,Arkansas,Arkansas,Arkansas,Arkansas,SEC,...,Away,5.5,74.0,95.0,-21.0,21.0,15.5,False,Home,False
1,2024-02-03,Saint Joseph's,Saint Joseph,STJOE,Saint Joseph's,Saint Joseph's,St Josephs,St. Joseph's,St Josephs,A10,...,Home,5.5,88.0,82.0,6.0,6.0,0.5,False,Away,False
2,2024-02-03,UConn,UConn,UConn,UConn,Connecticut,Connecticut,UConn,Connecticut,BE,...,Home,3.5,77.0,64.0,13.0,13.0,9.5,False,Away,False
3,2024-02-03,Penn St.,Penn State,PENST,Penn St.,Penn St.,Penn St,Penn State,Penn St,B10,...,Away,6.5,85.0,71.0,14.0,14.0,20.5,True,Away,True
4,2024-02-03,Duquesne,Duquesne,DUQ,Duquesne,Duquesne,Duquesne,Duquesne,Duquesne,A10,...,Home,5.5,85.0,71.0,14.0,14.0,8.5,False,Away,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,2024-02-14,South Carolina,South Carolina,SOCAR,South Carolina,South Carolina,South Carolina,South Carolina,S Carolina,SEC,...,Away,11.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,2024-02-14,Oregon St.,Oregon State,ORST,Oregon St.,Oregon St.,Oregon St,Oregon State,Oregon St,P12,...,Away,6.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,2024-02-14,UConn,UConn,UConn,UConn,Connecticut,Connecticut,UConn,Connecticut,BE,...,Home,24.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,2024-02-14,Tennessee,Tennessee,TENN,Tennessee,Tennessee,Tennessee,Tennessee,Tennessee,SEC,...,Home,8.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
line

,Team,Record,fanduel
0,Charleston So.,5-14,+8.5 -105 Open: +9.5
1,Winthrop,11-10,-8.5 -115 Open: -9.5
2,Georgetown,7-13,+12.5 -110 Open: +13.5
3,Seton Hall,14-8,-12.5 -110 Open: -13.5
4,Saint Louis,7-14,+4.5 -110 Open: +3.5
5,La Salle,10-11,-4.5 -110 Open: -3.5
6,High Point,18-4,-2.5 -102 Open: -2.5
7,UNC-Ash.,12-9,+2.5 -118 Open: +2.5
8,Stetson,11-10,-3.5 -115 Open: -2.5
9,Bellarmine,3-18,+3.5 -105 Open: +2.5
